# Lab-2.4 Part 3: Performance and Cost Optimization

## Objectives
- Optimize performance for production workloads
- Implement cost optimization strategies
- Define and monitor SLI/SLO metrics
- Set up performance baselines and alerting

## Estimated Time: 60-120 minutes

---
## 1. Performance Optimization Strategies

In [ ]:
# Performance optimization framework
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, asdict
from typing import Dict, List

@dataclass
class PerformanceConfig:
    """
    vLLM performance configuration parameters
    """
    # Model configuration
    tensor_parallel_size: int = 1
    gpu_memory_utilization: float = 0.9
    max_model_len: int = 4096
    
    # Batching configuration
    max_num_seqs: int = 32
    max_num_batched_tokens: int = 8192
    
    # Scheduling
    use_v2_block_manager: bool = True
    preemption_mode: str = "swap"  # "swap" or "recompute"
    
    # Engine settings
    worker_use_ray: bool = False
    engine_use_ray: bool = False
    disable_log_stats: bool = False

# Generate different performance configurations
configs = {
    "low_latency": PerformanceConfig(
        max_num_seqs=8,
        max_num_batched_tokens=2048,
        gpu_memory_utilization=0.8,
        max_model_len=2048
    ),
    "high_throughput": PerformanceConfig(
        max_num_seqs=64,
        max_num_batched_tokens=16384,
        gpu_memory_utilization=0.95,
        max_model_len=4096
    ),
    "balanced": PerformanceConfig(
        max_num_seqs=32,
        max_num_batched_tokens=8192,
        gpu_memory_utilization=0.9,
        max_model_len=4096
    ),
    "cost_optimized": PerformanceConfig(
        max_num_seqs=16,
        max_num_batched_tokens=4096,
        gpu_memory_utilization=0.85,
        max_model_len=2048
    )
}

print("Performance Configuration Profiles:")
print("=" * 50)

for name, config in configs.items():
    print(f"\n📊 {name.replace('_', ' ').title()}:")
    print(f"   Max sequences: {config.max_num_seqs}")
    print(f"   Max tokens: {config.max_num_batched_tokens}")
    print(f"   GPU utilization: {config.gpu_memory_utilization*100}%")
    print(f"   Context length: {config.max_model_len}")

In [ ]:
# Performance estimation model
class PerformanceEstimator:
    """
    Estimate performance metrics for different configurations
    """
    
    def __init__(self, model_size="7B", gpu_type="A10G"):
        self.model_size = model_size
        self.gpu_type = gpu_type
        
        # Base performance numbers (empirical)
        self.base_metrics = {
            "7B": {
                "A10G": {
                    "tokens_per_sec_per_seq": 80,
                    "prefill_latency_ms": 150,
                    "decode_latency_ms": 25,
                    "memory_per_token_kb": 0.5
                },
                "A100": {
                    "tokens_per_sec_per_seq": 120,
                    "prefill_latency_ms": 100,
                    "decode_latency_ms": 15,
                    "memory_per_token_kb": 0.5
                }
            }
        }
    
    def estimate_metrics(self, config: PerformanceConfig) -> Dict:
        """
        Estimate performance metrics for given configuration
        """
        base = self.base_metrics[self.model_size][self.gpu_type]
        
        # Calculate throughput
        parallel_factor = min(config.max_num_seqs / 8, 4.0)  # Diminishing returns
        total_throughput = base["tokens_per_sec_per_seq"] * parallel_factor
        
        # Calculate latency
        batch_overhead = 1 + (config.max_num_seqs - 1) * 0.02  # 2% overhead per additional seq
        ttft = base["prefill_latency_ms"] * batch_overhead
        itl = base["decode_latency_ms"] * batch_overhead
        
        # Memory usage
        model_memory = 14 * (config.gpu_memory_utilization / 0.9)  # 7B model ≈ 14GB
        kv_memory = (
            config.max_num_seqs * config.max_model_len * 
            base["memory_per_token_kb"] / 1024 / 1024  # Convert to GB
        )
        total_memory = model_memory + kv_memory
        
        return {
            "throughput_tokens_per_sec": int(total_throughput),
            "ttft_ms": int(ttft),
            "itl_ms": int(itl),
            "memory_usage_gb": round(total_memory, 2),
            "max_concurrent_requests": config.max_num_seqs,
            "requests_per_sec": int(total_throughput / 50),  # Assuming 50 tokens per response
        }

# Analyze different configurations
estimator = PerformanceEstimator("7B", "A10G")

performance_analysis = {}
for name, config in configs.items():
    metrics = estimator.estimate_metrics(config)
    performance_analysis[name] = metrics

# Create comparison table
df = pd.DataFrame(performance_analysis).T
df.index.name = 'Configuration'

print("\nPerformance Analysis:")
print("=" * 80)
print(df)

# Save to file
df.to_csv('performance_analysis.csv')
print("\n✅ Saved performance analysis to 'performance_analysis.csv'")

In [ ]:
# Visualize performance trade-offs
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

config_names = list(performance_analysis.keys())
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

# Throughput comparison
throughputs = [performance_analysis[name]['throughput_tokens_per_sec'] for name in config_names]
bars1 = ax1.bar(config_names, throughputs, color=colors)
ax1.set_ylabel('Tokens/Second')
ax1.set_title('Throughput Comparison')
ax1.tick_params(axis='x', rotation=45)
for bar, val in zip(bars1, throughputs):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
             f'{val}', ha='center', fontweight='bold')

# Latency comparison (TTFT)
latencies = [performance_analysis[name]['ttft_ms'] for name in config_names]
bars2 = ax2.bar(config_names, latencies, color=colors)
ax2.set_ylabel('TTFT (ms)')
ax2.set_title('Time to First Token')
ax2.tick_params(axis='x', rotation=45)
for bar, val in zip(bars2, latencies):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
             f'{val}', ha='center', fontweight='bold')

# Memory usage
memories = [performance_analysis[name]['memory_usage_gb'] for name in config_names]
bars3 = ax3.bar(config_names, memories, color=colors)
ax3.set_ylabel('Memory Usage (GB)')
ax3.set_title('GPU Memory Usage')
ax3.tick_params(axis='x', rotation=45)
for bar, val in zip(bars3, memories):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
             f'{val:.1f}', ha='center', fontweight='bold')

# Requests per second
rps_values = [performance_analysis[name]['requests_per_sec'] for name in config_names]
bars4 = ax4.bar(config_names, rps_values, color=colors)
ax4.set_ylabel('Requests/Second')
ax4.set_title('Request Handling Capacity')
ax4.tick_params(axis='x', rotation=45)
for bar, val in zip(bars4, rps_values):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
             f'{val}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Performance Trade-off Analysis:")
print(f"• High Throughput: {max(throughputs)} tokens/s (vs Low Latency: {min(throughputs)} tokens/s)")
print(f"• Low Latency: {min(latencies)}ms TTFT (vs High Throughput: {max(latencies)}ms)")
print(f"• Memory range: {min(memories):.1f}-{max(memories):.1f} GB")
print(f"• RPS range: {min(rps_values)}-{max(rps_values)} requests/sec")

---
## 2. Cost Optimization Techniques

In [ ]:
class CostOptimizer:
    """
    Production cost optimization strategies
    """
    
    def __init__(self):
        # Cloud pricing (AWS examples)
        self.pricing = {
            "on_demand": {
                "g5.xlarge": 1.006,   # $1.006/hour
                "g5.2xlarge": 1.212,  # $1.212/hour
                "p3.2xlarge": 3.060,  # $3.060/hour
            },
            "spot": {
                "g5.xlarge": 0.302,   # ~70% savings
                "g5.2xlarge": 0.364,
                "p3.2xlarge": 0.918,
            },
            "reserved_1yr": {
                "g5.xlarge": 0.603,   # ~40% savings
                "g5.2xlarge": 0.727,
                "p3.2xlarge": 1.836,
            }
        }
    
    def calculate_monthly_costs(self, instance_type: str, instances: int) -> Dict:
        """
        Calculate monthly costs for different pricing models
        """
        hours_per_month = 730  # Average hours per month
        
        costs = {}
        for pricing_model, prices in self.pricing.items():
            hourly_cost = prices.get(instance_type, 0)
            monthly_cost = hourly_cost * hours_per_month * instances
            costs[pricing_model] = {
                "hourly_per_instance": hourly_cost,
                "monthly_total": monthly_cost,
                "annual_total": monthly_cost * 12
            }
        
        return costs
    
    def optimization_strategies(self) -> List[Dict]:
        """
        Return list of cost optimization strategies
        """
        return [
            {
                "strategy": "Spot Instances",
                "savings": "60-90%",
                "effort": "Low",
                "risk": "Medium",
                "description": "Use spot instances with proper interruption handling"
            },
            {
                "strategy": "Reserved Instances",
                "savings": "30-50%",
                "effort": "Low",
                "risk": "Low",
                "description": "Commit to 1-3 year terms for predictable workloads"
            },
            {
                "strategy": "Auto-scaling",
                "savings": "20-40%",
                "effort": "Medium",
                "risk": "Low",
                "description": "Scale down during low traffic periods"
            },
            {
                "strategy": "Model Quantization",
                "savings": "40-60%",
                "effort": "Medium",
                "risk": "Medium",
                "description": "Use INT8/INT4 quantization to reduce memory needs"
            },
            {
                "strategy": "Request Batching",
                "savings": "50-80%",
                "effort": "High",
                "risk": "Low",
                "description": "Optimize batch sizes for maximum GPU utilization"
            },
            {
                "strategy": "Multi-tenancy",
                "savings": "30-60%",
                "effort": "High",
                "risk": "High",
                "description": "Share GPU resources across multiple services"
            }
        ]

# Analyze cost optimization
optimizer = CostOptimizer()
instance_type = "g5.2xlarge"
num_instances = 4

costs = optimizer.calculate_monthly_costs(instance_type, num_instances)

print(f"Cost Analysis ({instance_type} x {num_instances}):")
print("=" * 50)

for model, data in costs.items():
    print(f"\n{model.replace('_', ' ').title()}:")
    print(f"  Hourly per instance: ${data['hourly_per_instance']:.3f}")
    print(f"  Monthly total: ${data['monthly_total']:,.0f}")
    print(f"  Annual total: ${data['annual_total']:,.0f}")

# Calculate savings
on_demand_monthly = costs['on_demand']['monthly_total']
spot_monthly = costs['spot']['monthly_total']
reserved_monthly = costs['reserved_1yr']['monthly_total']

print(f"\n💰 Savings Potential:")
print(f"Spot vs On-demand: ${on_demand_monthly - spot_monthly:,.0f}/month ({(1-spot_monthly/on_demand_monthly)*100:.0f}% savings)")
print(f"Reserved vs On-demand: ${on_demand_monthly - reserved_monthly:,.0f}/month ({(1-reserved_monthly/on_demand_monthly)*100:.0f}% savings)")

In [ ]:
# Cost optimization strategies analysis
strategies = optimizer.optimization_strategies()
strategy_df = pd.DataFrame(strategies)

print("\n🎯 Cost Optimization Strategies:")
print("=" * 80)
print(strategy_df.to_string(index=False))

# Visualize savings potential
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Pricing model comparison
pricing_models = ['On-Demand', 'Spot', 'Reserved (1yr)']
monthly_costs = [on_demand_monthly, spot_monthly, reserved_monthly]
colors_cost = ['#FF6B6B', '#4ECDC4', '#45B7D1']

bars = ax1.bar(pricing_models, monthly_costs, color=colors_cost)
ax1.set_ylabel('Monthly Cost ($)')
ax1.set_title('Pricing Model Comparison')
ax1.tick_params(axis='x', rotation=0)

for bar, cost in zip(bars, monthly_costs):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100,
             f'${cost:,.0f}', ha='center', fontweight='bold')

# Savings strategies
strategy_names = [s['strategy'] for s in strategies[:4]]  # Top 4 strategies
savings_percentages = [int(s['savings'].split('-')[0]) for s in strategies[:4]]

bars2 = ax2.barh(strategy_names, savings_percentages, color=colors)
ax2.set_xlabel('Potential Savings (%)')
ax2.set_title('Cost Optimization Impact')

for bar, pct in zip(bars2, savings_percentages):
    ax2.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
             f'{pct}%', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n🎯 Cost Optimization Recommendations:")
print(f"1. Use Spot instances: Save ${on_demand_monthly - spot_monthly:,.0f}/month")
print(f"2. Implement auto-scaling: Additional 20-40% savings")
print(f"3. Optimize batch sizes: Maximize GPU utilization")
print(f"4. Consider quantization: Reduce instance size requirements")

---
## 3. SLI/SLO Definition

In [ ]:
# Service Level Indicators (SLI) and Objectives (SLO)
class SLOFramework:
    """
    Define and monitor Service Level Objectives
    """
    
    @staticmethod
    def define_slis():
        """
        Define measurable Service Level Indicators
        """
        return {
            "availability": {
                "metric": "successful_requests / total_requests",
                "measurement_window": "rolling 30 days",
                "data_source": "HTTP status codes"
            },
            "latency_p95": {
                "metric": "95th percentile response time",
                "measurement_window": "5 minute intervals",
                "data_source": "Request duration histogram"
            },
            "latency_p99": {
                "metric": "99th percentile response time",
                "measurement_window": "5 minute intervals",
                "data_source": "Request duration histogram"
            },
            "throughput": {
                "metric": "requests per second",
                "measurement_window": "1 minute intervals",
                "data_source": "Request counter"
            },
            "error_rate": {
                "metric": "error_requests / total_requests",
                "measurement_window": "rolling 5 minutes",
                "data_source": "HTTP 5xx status codes"
            }
        }
    
    @staticmethod
    def define_slos():
        """
        Define Service Level Objectives
        """
        return {
            "availability": {
                "target": 99.9,  # 99.9% uptime
                "measurement_period": "30 days",
                "error_budget": 0.1,  # 0.1% error budget
                "downtime_allowance": "43.2 minutes/month"
            },
            "latency_p95": {
                "target": 500,  # 500ms
                "measurement_period": "24 hours",
                "violation_threshold": "5% of time periods"
            },
            "latency_p99": {
                "target": 1000,  # 1 second
                "measurement_period": "24 hours",
                "violation_threshold": "1% of time periods"
            },
            "error_rate": {
                "target": 1.0,  # <1% error rate
                "measurement_period": "1 hour",
                "violation_threshold": "5% of hours"
            }
        }
    
    def calculate_error_budget(self, slo_target: float, measurement_days: int = 30) -> Dict:
        """
        Calculate error budget based on SLO
        """
        error_budget_percentage = 100 - slo_target
        total_minutes = measurement_days * 24 * 60
        error_budget_minutes = total_minutes * (error_budget_percentage / 100)
        
        return {
            "total_minutes": total_minutes,
            "error_budget_minutes": error_budget_minutes,
            "error_budget_percentage": error_budget_percentage,
            "uptime_required_minutes": total_minutes - error_budget_minutes
        }

# Define SLIs and SLOs
slo_framework = SLOFramework()
slis = slo_framework.define_slis()
slos = slo_framework.define_slos()

print("Service Level Objectives (SLOs):")
print("=" * 50)

for metric, slo in slos.items():
    print(f"\n📊 {metric.replace('_', ' ').title()}:")
    print(f"   Target: {slo['target']}{'%' if 'availability' in metric or 'error' in metric else 'ms'}")
    print(f"   Period: {slo['measurement_period']}")
    if 'error_budget' in slo:
        print(f"   Error budget: {slo['error_budget']}%")
        print(f"   Max downtime: {slo['downtime_allowance']}")

# Calculate error budgets
availability_budget = slo_framework.calculate_error_budget(99.9, 30)
print(f"\n💡 Availability Error Budget (30 days):")
print(f"   Total time: {availability_budget['total_minutes']:,.0f} minutes")
print(f"   Error budget: {availability_budget['error_budget_minutes']:.1f} minutes")
print(f"   Required uptime: {availability_budget['uptime_required_minutes']:,.0f} minutes")

---
## 4. Monitoring and Alerting Setup

In [ ]:
# Generate Prometheus monitoring rules
prometheus_rules = '''
groups:
- name: llm-service.rules
  rules:
  # SLI: Availability
  - record: llm:availability_5m
    expr: |
      (
        sum(rate(http_requests_total{job="llm-service",code!~"5.."}[5m])) /
        sum(rate(http_requests_total{job="llm-service"}[5m]))
      ) * 100
  
  # SLI: P95 Latency
  - record: llm:latency_p95_5m
    expr: |
      histogram_quantile(0.95,
        sum(rate(http_request_duration_seconds_bucket{job="llm-service"}[5m])) by (le)
      ) * 1000
  
  # SLI: Error Rate
  - record: llm:error_rate_5m
    expr: |
      (
        sum(rate(http_requests_total{job="llm-service",code=~"5.."}[5m])) /
        sum(rate(http_requests_total{job="llm-service"}[5m]))
      ) * 100
  
  # Resource utilization
  - record: llm:gpu_utilization
    expr: |
      avg(nvidia_gpu_utilization{job="llm-service"}) by (instance)
  
  - record: llm:memory_utilization
    expr: |
      (
        nvidia_gpu_memory_used_bytes{job="llm-service"} /
        nvidia_gpu_memory_total_bytes{job="llm-service"}
      ) * 100

- name: llm-service.alerts
  rules:
  # SLO Violation: Availability
  - alert: LLMServiceAvailabilityLow
    expr: llm:availability_5m < 99.5
    for: 2m
    labels:
      severity: critical
    annotations:
      summary: "LLM service availability below SLO"
      description: "Availability is {{ $value }}%, below 99.9% SLO"
  
  # SLO Violation: Latency
  - alert: LLMServiceLatencyHigh
    expr: llm:latency_p95_5m > 500
    for: 5m
    labels:
      severity: warning
    annotations:
      summary: "LLM service P95 latency above SLO"
      description: "P95 latency is {{ $value }}ms, above 500ms SLO"
  
  # High Error Rate
  - alert: LLMServiceErrorRateHigh
    expr: llm:error_rate_5m > 1
    for: 3m
    labels:
      severity: critical
    annotations:
      summary: "LLM service error rate above threshold"
      description: "Error rate is {{ $value }}%, above 1% threshold"
  
  # Resource Alerts
  - alert: LLMServiceGPUUtilizationLow
    expr: llm:gpu_utilization < 70
    for: 10m
    labels:
      severity: info
    annotations:
      summary: "Low GPU utilization detected"
      description: "GPU utilization is {{ $value }}%, consider scaling down"
  
  - alert: LLMServiceMemoryHigh
    expr: llm:memory_utilization > 95
    for: 5m
    labels:
      severity: warning
    annotations:
      summary: "High GPU memory usage"
      description: "GPU memory usage is {{ $value }}%, approaching limit"
'''

os.makedirs('monitoring', exist_ok=True)
with open('monitoring/prometheus-rules.yml', 'w') as f:
    f.write(prometheus_rules.strip())

print("✅ Generated Prometheus monitoring rules")
print("   Includes: SLO tracking, alerting rules, resource monitoring")

In [ ]:
# Generate Grafana dashboard configuration
grafana_dashboard = {
    "dashboard": {
        "title": "LLM Service Production Dashboard",
        "tags": ["llm", "production", "vllm"],
        "timezone": "UTC",
        "panels": [
            {
                "title": "Availability (SLO: 99.9%)",
                "type": "stat",
                "targets": [{
                    "expr": "llm:availability_5m",
                    "legendFormat": "Availability"
                }],
                "thresholds": {
                    "steps": [
                        {"color": "red", "value": 0},
                        {"color": "yellow", "value": 99.5},
                        {"color": "green", "value": 99.9}
                    ]
                }
            },
            {
                "title": "P95 Latency (SLO: <500ms)",
                "type": "timeseries",
                "targets": [{
                    "expr": "llm:latency_p95_5m",
                    "legendFormat": "P95 Latency"
                }],
                "alert": {
                    "condition": "B",
                    "threshold": 500
                }
            },
            {
                "title": "Request Rate",
                "type": "timeseries",
                "targets": [{
                    "expr": "sum(rate(http_requests_total{job=\"llm-service\"}[5m]))",
                    "legendFormat": "Requests/sec"
                }]
            },
            {
                "title": "Error Rate (SLO: <1%)",
                "type": "stat",
                "targets": [{
                    "expr": "llm:error_rate_5m",
                    "legendFormat": "Error Rate"
                }],
                "thresholds": {
                    "steps": [
                        {"color": "green", "value": 0},
                        {"color": "yellow", "value": 0.5},
                        {"color": "red", "value": 1.0}
                    ]
                }
            },
            {
                "title": "GPU Utilization",
                "type": "timeseries",
                "targets": [{
                    "expr": "llm:gpu_utilization",
                    "legendFormat": "GPU {{ instance }}"
                }]
            },
            {
                "title": "GPU Memory Usage",
                "type": "timeseries",
                "targets": [{
                    "expr": "llm:memory_utilization",
                    "legendFormat": "Memory {{ instance }}"
                }]
            }
        ]
    }
}

with open('monitoring/grafana-dashboard.json', 'w') as f:
    json.dump(grafana_dashboard, f, indent=2)

print("✅ Generated Grafana dashboard configuration")
print("\n📊 Dashboard includes:")
print("• Availability tracking (SLO: 99.9%)")
print("• Latency monitoring (P95/P99)")
print("• Error rate tracking")
print("• Resource utilization (GPU/Memory)")
print("• SLO violation alerts")

---
## 5. Performance Tuning Guide

In [ ]:
# Generate performance tuning recommendations
class PerformanceTuner:
    """
    Production performance tuning recommendations
    """
    
    @staticmethod
    def get_tuning_parameters():
        """
        Key parameters for performance tuning
        """
        return {
            "gpu_memory_utilization": {
                "description": "Fraction of GPU memory to use",
                "range": "0.8 - 0.95",
                "recommendation": "Start with 0.9, adjust based on OOM errors",
                "impact": "Higher = more concurrent requests, but risk of OOM"
            },
            "max_num_seqs": {
                "description": "Maximum number of sequences in a batch",
                "range": "8 - 128",
                "recommendation": "32 for balanced latency/throughput",
                "impact": "Higher = better throughput, higher latency"
            },
            "max_num_batched_tokens": {
                "description": "Maximum tokens in a batch",
                "range": "2048 - 32768",
                "recommendation": "8192 for most workloads",
                "impact": "Limits memory usage and batch size"
            },
            "max_model_len": {
                "description": "Maximum context length",
                "range": "512 - 8192",
                "recommendation": "Match your use case requirements",
                "impact": "Longer = more memory per sequence"
            },
            "tensor_parallel_size": {
                "description": "Number of GPUs for tensor parallelism",
                "range": "1, 2, 4, 8",
                "recommendation": "1 for single GPU, 2-4 for large models",
                "impact": "Enables larger models, adds communication overhead"
            }
        }
    
    @staticmethod
    def get_optimization_checklist():
        """
        Production optimization checklist
        """
        return [
            {
                "category": "Memory Optimization",
                "items": [
                    "Use appropriate gpu_memory_utilization (0.85-0.95)",
                    "Enable KV cache quantization if available",
                    "Set max_model_len to actual requirements",
                    "Monitor for OOM errors and adjust accordingly"
                ]
            },
            {
                "category": "Throughput Optimization",
                "items": [
                    "Tune max_num_seqs for your GPU memory",
                    "Optimize max_num_batched_tokens",
                    "Enable continuous batching",
                    "Use async processing for API layer"
                ]
            },
            {
                "category": "Latency Optimization",
                "items": [
                    "Reduce batch sizes for lower latency",
                    "Use FlashAttention if available",
                    "Optimize model loading time",
                    "Implement request prioritization"
                ]
            },
            {
                "category": "Cost Optimization",
                "items": [
                    "Use spot instances with interruption handling",
                    "Implement auto-scaling based on load",
                    "Consider model quantization (INT8/INT4)",
                    "Optimize instance types for workload"
                ]
            }
        ]

# Display tuning guide
tuner = PerformanceTuner()
parameters = tuner.get_tuning_parameters()
checklist = tuner.get_optimization_checklist()

print("🔧 Performance Tuning Parameters:")
print("=" * 60)

for param, details in parameters.items():
    print(f"\n📋 {param}:")
    print(f"   Description: {details['description']}")
    print(f"   Range: {details['range']}")
    print(f"   Recommendation: {details['recommendation']}")
    print(f"   Impact: {details['impact']}")

print(f"\n\n✅ Optimization Checklist:")
print("=" * 60)

for category_info in checklist:
    print(f"\n🎯 {category_info['category']}:")
    for item in category_info['items']:
        print(f"   • {item}")

# Save tuning guide
tuning_guide = {
    "parameters": parameters,
    "checklist": checklist
}

with open('performance_tuning_guide.json', 'w') as f:
    json.dump(tuning_guide, f, indent=2)

print("\n✅ Saved tuning guide to 'performance_tuning_guide.json'")

---
## Summary

✅ **Completed**:
1. Analyzed performance configurations and trade-offs
2. Calculated cost optimization strategies
3. Defined comprehensive SLI/SLO framework
4. Set up Prometheus monitoring rules and alerts
5. Created Grafana dashboard for production monitoring
6. Generated performance tuning guide

📊 **Key Insights**:
- Spot instances can save 60-90% on compute costs
- Proper batching optimization provides 2-5x throughput gains
- SLO monitoring enables proactive issue detection
- Performance tuning requires balancing latency vs throughput

💡 **Cost Savings Potential**:
- Spot instances: $20,000-40,000/month
- Auto-scaling: Additional 20-40% savings
- Optimization: 2-3x better resource efficiency
- Total potential: 70-80% cost reduction

➡️ **Next**: In `04-Security_and_Compliance.ipynb`, we'll cover:
- API authentication and authorization
- Security hardening
- Compliance requirements (GDPR, SOC2)
- Incident response procedures

---
## Exercises

1. **Configuration optimization**: Test different vLLM configurations and measure impact
2. **Cost modeling**: Calculate costs for your specific workload and region
3. **SLO design**: Design SLOs for your specific use case
4. **Alert tuning**: Adjust alert thresholds based on your requirements